
# Multi-Criteria Analysis (MCA) Lab — Net-Zero 2050 Policy Options (Updated)

This lab helps you explore how to evaluate policy pathways for achieving **net-zero emissions by 2050** using **Multi-Criteria Analysis (MCA)**.  
This notebook includes examples, weight visualization, and category-level analysis.

---

## Learning Goals
- Define decision alternatives and evaluation criteria
- Normalize heterogeneous indicators (0–100 scale)
- Apply different weighting strategies
- Create an aggregate index
- Visualize and interpret results
- Reflect on trade-offs and sensitivities

---


## 0) Setup — Install and import dependencies

In [ ]:

!pip -q install pandas numpy matplotlib openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option("display.max_colwidth", 120)



## 1) Define alternatives and criteria (Expanded Example)

We compare four policy alternatives on multiple technical, economic, environmental, social, and institutional criteria.


In [ ]:

# 🧩 Expanded dataset
raw_data = pd.DataFrame({
    "Alternative": [
        "NZ2050_DeepElec",
        "NZ2050_Balanced",
        "NPI_Current",
        "NZ2050_HydrogenPush"
    ],
    "Renewable_share_%": [85, 75, 55, 70],
    "Annual_storage_additions_GWh": [25, 18, 6, 22],
    "System_cost_pct_GDP": [4.2, 3.9, 3.2, 4.5],
    "Household_bill_EUR": [1200, 1300, 1500, 1250],
    "GHG_2050_MtCO2e": [30, 60, 220, 45],
    "Air_pollution_index": [20, 30, 60, 25],
    "Jobs_change_coal_regions_thousand": [+45, +30, -10, +25],
    "Energy_poverty_rate_%": [6, 7, 10, 8],
    "Implementation_readiness_score": [70, 65, 55, 68],
    "Regulatory_complexity_index": [30, 35, 50, 40],
    "Public_acceptance_index": [80, 70, 40, 60]
})

criterion_meta = pd.DataFrame({
    "criterion": [
        "Renewable_share_%",
        "Annual_storage_additions_GWh",
        "System_cost_pct_GDP",
        "Household_bill_EUR",
        "GHG_2050_MtCO2e",
        "Air_pollution_index",
        "Jobs_change_coal_regions_thousand",
        "Energy_poverty_rate_%",
        "Implementation_readiness_score",
        "Regulatory_complexity_index",
        "Public_acceptance_index"
    ],
    "direction": [
        "benefit","benefit","cost","cost","cost","cost",
        "benefit","cost","benefit","cost","benefit"
    ]
})

display(raw_data)
display(criterion_meta)


## 2) Normalization (0–100 scale)

In [ ]:

def normalize_minmax(df_raw, meta):
    df = df_raw.copy()
    df_norm = pd.DataFrame({"Alternative": df["Alternative"]})
    for _, row in meta.iterrows():
        c = row["criterion"]
        direction = row["direction"].strip().lower()
        values = df[c].astype(float)
        cmin, cmax = values.min(), values.max()
        if np.isclose(cmax, cmin):
            df_norm[c] = 100.0
            continue
        if direction == "benefit":
            df_norm[c] = 100.0 * (values - cmin) / (cmax - cmin)
        elif direction == "cost":
            df_norm[c] = 100.0 * (cmax - values) / (cmax - cmin)
        else:
            raise ValueError(f"Unknown direction for {c}")
    return df_norm

normalized = normalize_minmax(raw_data, criterion_meta)
normalized


## 3) Weighting the criteria (Updated Example)

In [ ]:

# ⚖️ Direct weights including Public_acceptance_index
direct_weights = {
    "Renewable_share_%": 0.08,
    "Annual_storage_additions_GWh": 0.07,
    "System_cost_pct_GDP": 0.12,
    "Household_bill_EUR": 0.08,
    "GHG_2050_MtCO2e": 0.25,
    "Air_pollution_index": 0.10,
    "Jobs_change_coal_regions_thousand": 0.10,
    "Energy_poverty_rate_%": 0.05,
    "Implementation_readiness_score": 0.05,
    "Regulatory_complexity_index": 0.03,
    "Public_acceptance_index": 0.07
}

crit_list = list(criterion_meta["criterion"])
w_direct = np.array([direct_weights[c] for c in crit_list])
w_sum = w_direct.sum()
if not np.isclose(w_sum, 1.0):
    w_direct /= w_sum

for c, w in zip(crit_list, w_direct):
    print(f"{c:35s} {w:.3f}")


### Visualize the weighting distribution

In [ ]:

weights_df = pd.DataFrame({"Criterion": crit_list, "Weight": w_direct}).sort_values("Weight", ascending=False)

plt.figure(figsize=(6,6))
plt.pie(weights_df["Weight"], labels=weights_df["Criterion"], autopct="%1.1f%%", startangle=140)
plt.title("Weight Distribution by Criterion")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8,5))
plt.barh(weights_df["Criterion"], weights_df["Weight"])
plt.xlabel("Weight (sum = 1.0)")
plt.title("MCA Criteria Weights — Visual Check")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


### Aggregate weights by category (pillar-level overview)

In [ ]:

category_map = {
    "Renewable_share_%": "Technical",
    "Annual_storage_additions_GWh": "Technical",
    "System_cost_pct_GDP": "Economic",
    "Household_bill_EUR": "Economic",
    "GHG_2050_MtCO2e": "Environmental",
    "Air_pollution_index": "Environmental",
    "Jobs_change_coal_regions_thousand": "Social",
    "Energy_poverty_rate_%": "Social",
    "Implementation_readiness_score": "Institutional",
    "Regulatory_complexity_index": "Institutional",
    "Public_acceptance_index": "Social"
}

cat_df = pd.DataFrame({
    "Criterion": crit_list,
    "Weight": w_direct,
    "Category": [category_map.get(c, "Other") for c in crit_list]
})
cat_summary = cat_df.groupby("Category", as_index=False)["Weight"].sum().sort_values("Weight", ascending=False)

display(cat_summary)

plt.figure(figsize=(7,4))
plt.bar(cat_summary["Category"], cat_summary["Weight"])
plt.title("Aggregate Weights by Category (Pillar-Level)")
plt.ylabel("Weight share (sum=1.0)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(6,6))
plt.pie(cat_summary["Weight"], labels=cat_summary["Category"], autopct="%1.1f%%", startangle=140)
plt.title("Category Weight Distribution")
plt.tight_layout()
plt.show()


## 4) Aggregate index — Weighted Sum


---

## 🧭 Applying MCA at Different Levels: Scenarios vs. Technologies

Depending on your available data and the question you're addressing, you can apply this MCA framework in two complementary ways:

### **Path A — Scenario-Level Analysis**
Use this when comparing *integrated policy pathways* (e.g., "Net-Zero 2050", "Current Policies", "Hydrogen Push").
- Each alternative represents a **comprehensive scenario** from an energy system model.
- Criteria aggregate system-wide outcomes (emissions, cost, jobs, energy poverty, etc.).
- Useful for **policy recommendations** at the strategic level — e.g., “Which national pathway aligns best with net-zero and social goals?”

### **Path B — Technology-Level Analysis**
Use this when scenario outputs are too similar or when you want to explore **power-sector composition** in more detail.
- Alternatives represent **technologies** or **project options** (e.g., "Onshore Wind", "Solar PV", "Gas + CCS").
- Criteria describe performance, cost, environmental impact, land use, public acceptance, etc.
- Suitable for exploring **realism** and **trade-offs** in specific country contexts (e.g., Malta’s land constraints vs. solar potential).

> 🧠 **In practice:** Students may choose either or both approaches.  
> The key is to maintain a transparent link between **quantitative model evidence** and **qualitative reasoning** (social, environmental, and institutional dimensions).



## 5) Comparing results under two different weight sets

To understand **how sensitive your MCA rankings are to stakeholder priorities**, you can test different sets of weights.

In this example, we compare:
- **Environmental Priority:** GHG and Air Pollution criteria get higher weights.
- **Economic Priority:** System cost and household bills are prioritized.

This helps visualize how the ranking of alternatives shifts when the emphasis changes.


In [ ]:

# Define two different weighting schemes
weights_env_priority = {
    "Renewable_share_%": 0.08,
    "Annual_storage_additions_GWh": 0.06,
    "System_cost_pct_GDP": 0.05,
    "Household_bill_EUR": 0.05,
    "GHG_2050_MtCO2e": 0.35,
    "Air_pollution_index": 0.15,
    "Jobs_change_coal_regions_thousand": 0.07,
    "Energy_poverty_rate_%": 0.04,
    "Implementation_readiness_score": 0.07,
    "Regulatory_complexity_index": 0.03,
    "Public_acceptance_index": 0.05
}

weights_econ_priority = {
    "Renewable_share_%": 0.06,
    "Annual_storage_additions_GWh": 0.05,
    "System_cost_pct_GDP": 0.25,
    "Household_bill_EUR": 0.20,
    "GHG_2050_MtCO2e": 0.15,
    "Air_pollution_index": 0.05,
    "Jobs_change_coal_regions_thousand": 0.07,
    "Energy_poverty_rate_%": 0.05,
    "Implementation_readiness_score": 0.07,
    "Regulatory_complexity_index": 0.03,
    "Public_acceptance_index": 0.02
}

# Convert to aligned arrays
crit_list = list(criterion_meta["criterion"])
w_env = np.array([weights_env_priority[c] for c in crit_list])
w_econ = np.array([weights_econ_priority[c] for c in crit_list])

# Ensure normalization
w_env = w_env / w_env.sum()
w_econ = w_econ / w_econ.sum()

def aggregate_weighted_sum(df_norm, meta, w):
    crit = list(meta["criterion"])
    scores = df_norm[crit].values
    index = (scores * w).sum(axis=1)
    out = df_norm[["Alternative"]].copy()
    out["MCA_WeightedSum_0to100"] = index
    return out

# Aggregate results
agg_env = aggregate_weighted_sum(normalized, criterion_meta, w_env)
agg_env.rename(columns={"MCA_WeightedSum_0to100": "Env_Priority_Score"}, inplace=True)

agg_econ = aggregate_weighted_sum(normalized, criterion_meta, w_econ)
agg_econ.rename(columns={"MCA_WeightedSum_0to100": "Econ_Priority_Score"}, inplace=True)

# Merge and rank
compare_df = agg_env.merge(agg_econ, on="Alternative")
compare_df["Rank_Env"] = compare_df["Env_Priority_Score"].rank(ascending=False, method="min").astype(int)
compare_df["Rank_Econ"] = compare_df["Econ_Priority_Score"].rank(ascending=False, method="min").astype(int)

display(compare_df.sort_values("Rank_Env"))


In [ ]:

# Visualize ranking shifts between the two weighting schemes
plt.figure(figsize=(7,5))
for i, row in compare_df.iterrows():
    plt.plot(["Environmental Priority", "Economic Priority"],
             [row["Rank_Env"], row["Rank_Econ"]],
             marker="o", label=row["Alternative"])
plt.gca().invert_yaxis()
plt.title("Ranking Shifts under Different Weighting Schemes")
plt.ylabel("Rank (1 = best)")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:

# 📊 ΔRank Sensitivity Summary
# Calculate how much each alternative's rank changes between the two weighting schemes
compare_df["Rank_Change_(abs)"] = (compare_df["Rank_Env"] - compare_df["Rank_Econ"]).abs()
compare_df_sorted = compare_df.sort_values("Rank_Change_(abs)", ascending=False)

print("Ranking sensitivity between Environmental and Economic priority weightings:")
display(compare_df_sorted[["Alternative", "Rank_Env", "Rank_Econ", "Rank_Change_(abs)"]])



> 💬 **How to interpret ΔRank**
>
> - A **high ΔRank** means the option is *sensitive to stakeholder priorities* — it performs well only under specific value systems.
> - A **low ΔRank** means the option is *robust* — it ranks consistently across different perspectives.
> - In your report, discuss whether your recommended alternative is *robust* or *priority-dependent*, and what this implies for policymaking and public dialogue.



> 🧠 **Interpretation:**
>
> - Large ranking shifts indicate **sensitivity to priorities** — policies may be preferred only under certain value systems.
> - Stable rankings suggest **robust** options that perform well across stakeholder perspectives.
> - Encourage students to discuss which weighting scenario reflects their country's policy stance or public opinion.


In [ ]:

def aggregate_weighted_sum(df_norm, meta, w):
    crit = list(meta["criterion"])
    scores = df_norm[crit].values
    index = (scores * w).sum(axis=1)
    out = df_norm[["Alternative"]].copy()
    out["MCA_WeightedSum_0to100"] = index
    return out

agg_ws = aggregate_weighted_sum(normalized, criterion_meta, w_direct)
agg_ws.sort_values("MCA_WeightedSum_0to100", ascending=False)
